In [1]:
#Load the imports
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from matplotlib import pyplot as plt
from datetime import datetime
from sklearn.model_selection import train_test_split
# The following lines adjust the granularity of reporting.
pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.1f}".format
print("Imported modules.")

Imported modules.


In [2]:
#Define program constants

DATABASE_PATH = "database.csv"
SEQUENCE_LEN = 26 #26 points equals 26*15 days equals a year aprox.

In [3]:
# Load the dataset
aux_df = pd.read_csv(DATABASE_PATH)
scale_factor = 1000.0
# Shuffle the examples
#train_df = train_df.reindex(np.random.permutation(train_df.index))
#print(aux_df[['date']])

In [4]:

def convert_to_timestamp(date_str):

    if(isinstance(aux_df['date'][0], np.int64)):
      print("Already converted")
      return False #NOT ROBUST, FAILS IF RUN TWICE ON THE SAME DF

    date_obj = datetime.strptime(date_str, "%m/%d/%Y")
    return int(date_obj.timestamp())# * 1000) Optional multiplication by 1000 to turn into miliseconds.


aux_df['date'] = aux_df['date'].apply(convert_to_timestamp)

In [5]:
#Check if dates are correctly turned into a timsetamp format
print(aux_df['date'][0])
date = datetime.fromtimestamp(aux_df['date'][0])
year = date.year
month = date.month
day = date.day

print(year, month, day)

978318000
2001 1 1


In [6]:
#divide the dataframe into three sections (train, test, validation) with roughly a 70-20-10 distribution
#The samples are divided secuentially to improve the RNN performance

train_df, test_df = train_test_split(aux_df, test_size=0.2, shuffle=False)  # 80% to training df and 20% to testing df
train_df, val_df = train_test_split(train_df, test_size=0.1, shuffle=False)  # Of the 80%, 90% goes to training and 10% to validation

# To make each df legth divisible by the batches in the model building section
train_df = train_df[:-(len(train_df) % SEQUENCE_LEN)]
val_df = val_df[:-(len(val_df) % SEQUENCE_LEN)]
test_df = test_df[:-(len(test_df) % SEQUENCE_LEN)]


DATA NORMALIZATION INFO

https://medium.com/@spinjosovsky/normalize-data-before-or-after-split-of-training-and-testing-data-7b8005f81e26
https://datascience.stackexchange.com/questions/27615/should-we-apply-normalization-to-test-data-as-well

In [7]:
# Normalization of every dataframe. The chosen normalization uses mean and standard deviation
# Every Dataframe must be normalized by the same values, hence the train df mean and deviation are saved
train_mean = train_df.mean()
train_std = train_df.std()


def normalize_data(data, mean, std):
    return (data - mean) / std

#The id field shouldn't be normalized yet is not used in the model
train_normalized = normalize_data(train_df, train_mean, train_std)
val_normalized = normalize_data(val_df, train_mean, train_std)
test_normalized = normalize_data(test_df, train_mean, train_std)

In [26]:
# Keras Input tensors of float values.
#The prediction is going to use one value for lat, long, precipitation, temperature (the ones on the date desired to estimate)
#and the SEQUENCE_LEN previous values of ppna
inputs = {
    'date':
        tf.keras.layers.Input(shape=(1,), dtype=tf.float32,
                              name='date'),
    'latitude':
        tf.keras.layers.Input(shape=(1,), dtype=tf.float32,
                              name='latitude'),
    'longitude':
        tf.keras.layers.Input(shape=(1,), dtype=tf.float32,
                              name='longitude'),
    'ppna':
        tf.keras.layers.Input(shape=(SEQUENCE_LEN, 1), dtype=tf.float32,
                             name='ppna'),
    'precipitation':
        tf.keras.layers.Input(shape=(1,), dtype=tf.float32,
                              name='precipitation'),
    'temperature':
        tf.keras.layers.Input(shape=(1,), dtype=tf.float32,
                              name='temperature')
}

In [27]:
# Define the model for the NN
# LSTM layer with 64 nodes processing the ppna -- the only parameter that takes info from the past
lstm_layer = tf.keras.layers.LSTM(64)(inputs['ppna'])

# Concatenate the inputs and LSTM output to provide the model tools to learn the patterns
concatenated_features = tf.keras.layers.concatenate([inputs['date'], inputs['latitude'], inputs['longitude'], inputs['precipitation'], inputs['temperature'], lstm_layer])

# Output Layer, with just one output (Estimated PPNA) that receives the concatenated features
output = tf.keras.layers.Dense(1, name='ppna_output')(concatenated_features)

# Create the model
model = tf.keras.Model(inputs=inputs, outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae', 'accuracy'])
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 ppna (InputLayer)           [(None, 26, 1)]              0         []                            
                                                                                                  
 date (InputLayer)           [(None, 1)]                  0         []                            
                                                                                                  
 latitude (InputLayer)       [(None, 1)]                  0         []                            
                                                                                                  
 longitude (InputLayer)      [(None, 1)]                  0         []                            
                                                                                            

In [28]:
#Model training

#Define the training parameters
epochs = 5  
batch_size = 64 

#Define the features and labels for each section of the process.
#The label is the ppna (what we want to predict)
#The features are the values used to predict said label (previous ppna, precipitations, temperature, lat, long, date)
train_labels = train_normalized['ppna'] 
train_features = {
        'date': train_normalized['date'],
        'latitude': train_normalized['latitude'],
        'longitude': train_normalized['longitude'],
        'ppna': train_normalized['ppna'],
        'precipitation': train_normalized['ppt'],
        'temperature': train_normalized['temp']
}

val_labels = val_normalized['ppna'] 
val_features = {
         'date': val_normalized['date'],
         'latitude': val_normalized['latitude'],
         'longitude': val_normalized['longitude'],
         'ppna': val_normalized['ppna'],
         'precipitation': val_normalized['ppt'],
         'temperature': val_normalized['temp']
}

test_labels = test_normalized['ppna']
test_features = {
    'date': test_normalized['date'],
    'latitude': test_normalized['latitude'],
    'longitude': test_normalized['longitude'],
    'ppna': test_normalized['ppna'],
    'precipitation': test_normalized['ppt'],
    'temperature': test_normalized['temp']
}

#Train the model with the previously defined parameters and data
history = model.fit(
    train_features, 
    train_labels, 
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(val_features, val_labels)  
)

# Evaluate the model
test_loss = model.evaluate(test_features, test_labels)  

print("Pérdida en el conjunto de prueba:", test_loss)

Epoch 1/5
7763/7763 [==============================] - 71s 7ms/step - loss: 0.0073 - mae: 0.0187 - accuracy: 0.0000e+00 - val_loss: 3.2754e-06 - val_mae: 0.0013 - val_accuracy: 0.0000e+00
Epoch 2/5
7763/7763 [==============================] - 57s 7ms/step - loss: 3.3021e-06 - mae: 0.0011 - accuracy: 0.0000e+00 - val_loss: 2.4426e-06 - val_mae: 0.0013 - val_accuracy: 0.0000e+00
Epoch 3/5
7763/7763 [==============================] - 51s 7ms/step - loss: 2.4075e-06 - mae: 8.9188e-04 - accuracy: 0.0000e+00 - val_loss: 6.5731e-07 - val_mae: 4.8099e-04 - val_accuracy: 0.0000e+00
Epoch 4/5
7763/7763 [==============================] - 64s 8ms/step - loss: 1.9573e-06 - mae: 8.0809e-04 - accuracy: 0.0000e+00 - val_loss: 4.5341e-07 - val_mae: 4.0556e-04 - val_accuracy: 0.0000e+00
Epoch 5/5
4312/4312 [==============================] - 26s 6ms/step - loss: 8.4605e-07 - mae: 6.5426e-04 - accuracy: 0.0000e+00
Pérdida en el conjunto de prueba: [8.460481240035733e-07, 0.0006542647606693208, 0.0]


DE ACA PARA ABAJO SOLO HAY CODIGO SIN USAR, PARTES SIN PROBAR QUE GUARDE POR SI FALLA LO QUE HICE. NO EJECUTAR

In [ ]:

# Define the model for the NN
# CODIGO ALTERNATIVO PARA CREAR MODELO. NO EJECUTAR
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
num_caracteristicas = 6  # Número de características: PPNA, temperatura, precipitaciones, latitud, longitud, fecha

# Modelo RNN con capa LSTM
model = models.Sequential()

# Agrega una capa LSTM con return_sequences=True para mantener las secuencias
model.add(layers.LSTM(units=64, return_sequences=True, input_shape=(SEQUENCE_LEN, num_caracteristicas)))
# Agrega otra capa LSTM si es necesario
model.add(layers.LSTM(units=64, return_sequences=False))  # False para la última capa LSTM

# Capa de salida para predecir la PPNA futura
model.add(layers.Dense(1))

# Compila el modelo
model.compile(loss='mean_squared_error', optimizer='adam')

# Resumen del modelo
model.summary()


In [17]:
#Model training NO EJECUTAR, CODIGO BORRADOR

epochs = 1  
batch_size = 128  # Tamaño del lote de entrenamiento (ajusta según sea necesario)
train_labels = train_normalized['ppna']
val_labels = val_normalized['ppna']

test_labels = test_normalized['ppna']
test_features = {
    'latitude': test_normalized['latitude'],
    'longitude': test_normalized['longitude'],
    'ppna': test_normalized['ppna'],
    'precipitation': test_normalized['ppt'],
    'temperature': test_normalized['temp']
}

history = model.fit(
    x={'latitude': train_normalized['latitude'],
       'longitude': train_normalized['longitude'],
       'ppna': train_normalized['ppna'],
       'precipitation': train_normalized['ppt'],
       'temperature': train_normalized['temp']},
    y=train_labels,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(
        {'latitude': val_normalized['latitude'],
         'longitude': val_normalized['longitude'],
         'ppna': val_normalized['ppna'],
         'precipitation': val_normalized['ppt'],
         'temperature': val_normalized['temp']},
        val_labels
    )
)

# Evalúa el modelo en el conjunto de prueba
test_loss = model.evaluate(test_features, test_labels)  # Reemplaza test_labels por las etiquetas reales

# Imprime el resultado de la evaluación
print("Pérdida en el conjunto de prueba:", test_loss)

3882/3882 [==============================] - 1008s 260ms/step - loss: 6.7000e-06 - mae: 0.0015 - val_loss: 2.8992e-06 - val_mae: 9.4896e-04


ValueError: Layer "model_2" expects 5 input(s), but it received 1 input tensors. Inputs received: [<tf.Tensor: shape=(32, 7), dtype=float64, numpy=
array([[ 2.1170918 , -1.72874062, -0.26278911, -1.0814139 ,  0.82817595,
        -0.20319931,  1.27145415],
       [ 2.11709878, -1.72115296, -0.26278911, -1.0814139 ,  0.55131022,
         0.05242548,  1.11048984],
       [ 2.11710575, -1.7135653 , -0.26278911, -1.0814139 ,  0.17709505,
         0.34331331,  1.1605987 ],
       [ 2.11711272, -1.70597764, -0.26278911, -1.0814139 ,  0.03009318,
         0.04313373,  1.61463614],
       [ 2.1171197 , -1.69838998, -0.26278911, -1.0814139 , -0.18994747,
         0.21244649,  1.18319312],
       [ 2.11712667, -1.69080232, -0.26278911, -1.0814139 , -0.63195058,
         0.17178716,  0.20023021],
       [ 2.11713364, -1.68321466, -0.26278911, -1.0814139 , -0.57984389,
         0.49953076,  0.14986643],
       [ 2.11714062, -1.675627  , -0.26278911, -1.0814139 , -0.9956881 ,
         0.11779576, -0